In [18]:
from itertools import combinations
from pprint import pprint as pp

def generate_input(attrs_by_obj):
    objs_by_attr = dict()
    for obj, attrs in attrs_by_obj.iteritems():
        for attr in attrs:
            if attr not in objs_by_attr:
                objs_by_attr[attr] = set()
            objs_by_attr[attr].add(obj)
    
    attributes = reduce(lambda a, b: a.union(b), attrs_by_obj.values())
    objects = sorted(attrs_by_obj.keys())
    
    attributes_set = set(attributes)
    objects_set = set(objects)
    
    return attributes, attributes_set, objs_by_attr, objects, objects_set, attrs_by_obj
    

def obj_closure(objects, all_attributes, attrs_by_obj):
    m = set(all_attributes)
    for obj in objects:
        m = m.intersection(attrs_by_obj[obj])
    return m

def attr_closure(attributes, all_objects, objs_by_attr):
    g = set(all_objects)
    for attr in attributes:
        g = g.intersection(objs_by_attr[attr])
    return g


def closure(attributes, all_attributes, all_objects, attrs_by_obj, objs_by_attr):
    return obj_closure(attr_closure(attributes, all_objects, objs_by_attr), all_attributes, attrs_by_obj)


def get_preclosed_intents():
    pseudo_intents = list()
    
    for i in xrange(0, len(attributes_set) + 1):
        for subset in list(combinations(attributes_set, i)):
            subset = set(subset)
            is_pseudo_intent = True
            for pseudo_intent in pseudo_intents:
                if pseudo_intent < subset:
                    if not closure(pseudo_intent) <= subset:
                        is_pseudo_intent = False
                        break
            if is_pseudo_intent:
                pseudo_intents.append(subset)
    return pseudo_intents


def get_pseudo_intents(attributes_set, get_closure):
    pseudo_intents = list()
    
    for i in xrange(0, len(attributes_set) + 1):
        for subset in list(combinations(attributes_set, i)):
            subset = set(subset)
            if subset != get_closure(subset):
                is_pseudo_intent = True
                for pseudo_intent in pseudo_intents:
                    if pseudo_intent < subset:
                        if not get_closure(pseudo_intent) <= subset:
                            is_pseudo_intent = False
                            break
                if is_pseudo_intent:
                    pseudo_intents.append(subset)
    return pseudo_intents

def get_preclosure(intent_set, pseudo_intents, get_closure):
    prev_intents = None
    cur_intents = intent_set
    while prev_intents != cur_intents:
        pseudo_intents_union = set()
        for pseudo_intent in pseudo_intents:
            if pseudo_intent < cur_intents:
                pseudo_intents_union = pseudo_intents_union.union(get_closure(pseudo_intent))
        prev_intents = cur_intents
        cur_intents = cur_intents.union(pseudo_intents_union)
    return cur_intents

def next_closure(attribute_subset, attributes, get_closure):
    s = set(attribute_subset)
    for a in reversed(attributes):
        if a in s:
            s = s.difference(set([a]))
        else:
            b = get_closure(s.union(set([a])))
            diff = b.difference(s)
            contains_smaller = any(attributes.index(i) < attributes.index(a) for i in diff)
            if not contains_smaller:
                return b
    return set(attributes)

def get_canonical_basis(attributes, get_closure, get_preclosure):
    el = []
    a = set()
    attributes_set = set(attributes)
    while a != attributes_set:
        a_closure = get_closure(a) 
        if a != a_closure:
            el.append([a, '->', a_closure.difference(a)])
        a = next_closure(a, attributes, get_preclosure)
    return el


def print_canonical_basis(attrs_by_obj):
    all_attributes, all_attributes_set, objs_by_attr, all_objects, all_objects_set, attrs_by_obj = generate_input(attrs_by_obj)
    
    def get_closure(attributes):
        return closure(attributes, all_attributes, all_objects, attrs_by_obj, objs_by_attr)
        
    pseudo_intents = get_pseudo_intents(all_attributes_set, get_closure)
    preclosure = lambda intent_set: get_preclosure(intent_set, pseudo_intents, get_closure)
    basis = get_canonical_basis(sorted(all_attributes), get_closure, preclosure)
    pp(len(basis))
    pp(basis)

In [19]:
# Week 4 example 
attrs_by_obj_example = {
    '1': set(['a', 'c', 'd']),
    '2': set(['a', 'c']),
    '3': set(['b', 'c']),
    '4': set(['b', 'd']),
}
print_canonical_basis(attrs_by_obj_example)

3
[[set(['c', 'd']), '->', set(['a'])],
 [set(['a']), '->', set(['c'])],
 [set(['a', 'b', 'c']), '->', set(['d'])]]


In [20]:
# Week 4 quiz 2 question 1
attrs_by_obj_question_1 = {
    '1': set(['b', 'd']),
    '2': set(['b', 'e']),
    '3': set(['c']),
    '4': set(['a', 'b', 'c']),
    '5': set(['d']),
    '6': set(['b', 'c']),
    '7': set(['e']),
    }

print_canonical_basis(attrs_by_obj_question_1)   

4
[[set(['d', 'e']), '->', set(['a', 'b', 'c'])],
 [set(['c', 'e']), '->', set(['a', 'b', 'd'])],
 [set(['c', 'd']), '->', set(['a', 'b', 'e'])],
 [set(['a']), '->', set(['b', 'c'])]]


In [23]:
# Week 4 quiz 2 question 2
attrs_by_obj_question_2 = {
    '1': set(['b', 'c', 'e', 'f']),
    '2': set(['b', 'c']),
    '3': set(['a', 'b', 'c', 'd', 'e', 'f']),
    '4': set(['e', 'f']),
    '5': set(['c', 'e', 'f']),
    '6': set(['b']),
    '7': set(['c', 'e', 'f']),
    '8': set(['c', 'd', 'f']),
    '9': set(['c', 'e', 'f']),
    '10': set(['b', 'c', 'd', 'e', 'f'])
}

print_canonical_basis(attrs_by_obj_question_2)

5
[[set(['e']), '->', set(['f'])],
 [set(['d']), '->', set(['c', 'f'])],
 [set(['c', 'd', 'e', 'f']), '->', set(['b'])],
 [set(['b', 'f']), '->', set(['c', 'e'])],
 [set(['a']), '->', set(['b', 'c', 'd', 'e', 'f'])]]


In [24]:
# Week 4 quiz 2 question 3
attrs_by_obj_question_3 = {
    '1': set(['a', 'c', 'f']),
    '2': set(['b', 'd', 'f']),
    '3': set(['b', 'c', 'e', 'f']),
    '4': set(['b', 'c', 'f']),
    '5': set(['d', 'f']),
    '6': set(['a', 'b', 'c', 'e', 'f']),
    '7': set(['b', 'd', 'f'])
}
print_canonical_basis(attrs_by_obj_question_3)

5
[[set([]), '->', set(['f'])],
 [set(['e', 'f']), '->', set(['b', 'c'])],
 [set(['c', 'd', 'f']), '->', set(['a', 'b', 'e'])],
 [set(['a', 'f']), '->', set(['c'])],
 [set(['a', 'b', 'c', 'f']), '->', set(['e'])]]


In [26]:
# Week 4 quiz 2 question 4
attrs_by_obj_question_4 = {
    '1': set(['a', 'b', 'c', 'd', 'e', 'g', 'j', 'k']),
    '2': set(['b', 'd', 'k']),
    '3': set(['b', 'd', 'k']),
    '4': set(['d']),
    '5': set(['b', 'c', 'd', 'e', 'f', 'k']),
    '6': set(['b']),
    '7': set(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'k']),
    '8': set(['a', 'c', 'g', 'j', 'k']),
    '9': set(['a', 'b', 'd', 'f', 'k']),
    '10': set(['b', 'c', 'd', 'e', 'f', 'k']),
    '11': set(['a', 'b', 'd', 'j', 'k']),
    '12': set(['a', 'b', 'c', 'd', 'g', 'j', 'k']),
    '13': set(['a', 'b', 'd', 'f', 'g', 'k']),
}
print_canonical_basis(attrs_by_obj_question_4)

13
[[set(['j']), '->', set(['a', 'k'])],
 [set(['g']), '->', set(['a', 'k'])],
 [set(['f']), '->', set(['b', 'd', 'k'])],
 [set(['e']), '->', set(['b', 'c', 'd', 'k'])],
 [set(['d', 'k']), '->', set(['b'])],
 [set(['c']), '->', set(['k'])],
 [set(['b', 'k']), '->', set(['d'])],
 [set(['b', 'd']), '->', set(['k'])],
 [set(['b', 'c', 'd', 'f', 'k']), '->', set(['e'])],
 [set(['a']), '->', set(['k'])],
 [set(['a', 'g', 'j', 'k']), '->', set(['c'])],
 [set(['a', 'c', 'k']), '->', set(['g'])],
 [set(['a', 'b', 'd', 'f', 'j', 'k']), '->', set(['c', 'e', 'g'])]]


In [27]:
# Week 4 quiz 2 question 4
attrs_by_obj_context_1 = {
    '1': set(['a', 'd']),
    '2': set(['a', 'b', 'c', 'd']),
    '3': set(['c']),
    '4': set(['b', 'd']),
}
print_canonical_basis(attrs_by_obj_context_1)

4
[[set(['c', 'd']), '->', set(['a', 'b'])],
 [set(['b']), '->', set(['d'])],
 [set(['a']), '->', set(['d'])],
 [set(['a', 'b', 'd']), '->', set(['c'])]]


In [28]:
# Week 4 quiz 2 question 4
attrs_by_obj_context_2 = {
    '1': set(['a', 'd']),
    '2': set(['a', 'b', 'c', 'd']),
    '3': set(['b', 'd'])
}
print_canonical_basis(attrs_by_obj_context_2)

3
[[set([]), '->', set(['d'])],
 [set(['c', 'd']), '->', set(['a', 'b'])],
 [set(['a', 'b', 'd']), '->', set(['c'])]]


In [29]:
# Week 4 quiz 2 question 4
attrs_by_obj_context_3 = {
    '1': set(['a', 'd']),
    '2': set(['c']),
    '3': set(['b', 'd'])
}
print_canonical_basis(attrs_by_obj_context_3)

4
[[set(['c', 'd']), '->', set(['a', 'b'])],
 [set(['b']), '->', set(['d'])],
 [set(['a']), '->', set(['d'])],
 [set(['a', 'b', 'd']), '->', set(['c'])]]


In [30]:
# Week 4 quiz 2 question 4
attrs_by_obj_context_4 = {
    '1': set(['a', 'd']),
    '2': set(['a', 'b', 'c', 'd']),
    '3': set(['c']),
    '4': set(['b', 'd']),
    '5': set(['d']),
}
print_canonical_basis(attrs_by_obj_context_4)

4
[[set(['c', 'd']), '->', set(['a', 'b'])],
 [set(['b']), '->', set(['d'])],
 [set(['a']), '->', set(['d'])],
 [set(['a', 'b', 'd']), '->', set(['c'])]]


In [31]:
# Week 4 quiz 2 question 4
attrs_by_obj_context_5 = {
    '1': set(['a', 'd']),
    '2': set(['d']),
    '3': set(['c']),
    '4': set(['b', 'd'])
}
print_canonical_basis(attrs_by_obj_context_5)

4
[[set(['c', 'd']), '->', set(['a', 'b'])],
 [set(['b']), '->', set(['d'])],
 [set(['a']), '->', set(['d'])],
 [set(['a', 'b', 'd']), '->', set(['c'])]]


In [32]:
# Week 4 quiz 2 question 4
attrs_by_obj_context_6 = {
    '1': set(['a', 'd']),
    '2': set(['a', 'b', 'c']),
    '3': set(['c']),
    '4': set(['b', 'd'])
}
print_canonical_basis(attrs_by_obj_context_6)

4
[[set(['c', 'd']), '->', set(['a', 'b'])],
 [set(['b', 'c']), '->', set(['a'])],
 [set(['a', 'c']), '->', set(['b'])],
 [set(['a', 'b']), '->', set(['c'])]]


In [33]:
# Week 4 quiz 2 question 4
attrs_by_obj_context_7 = {
    '1': set(['a', 'd']),
    '2': set([]),
    '3': set(['c']),
    '4': set(['b', 'd'])
}
print_canonical_basis(attrs_by_obj_context_7)

4
[[set(['c', 'd']), '->', set(['a', 'b'])],
 [set(['b']), '->', set(['d'])],
 [set(['a']), '->', set(['d'])],
 [set(['a', 'b', 'd']), '->', set(['c'])]]
